In [2]:
from functools import partial
from shiny import App, ui, render
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# read in data
df = pd.read_csv("C:/Users/sienn/Downloads/matchscouting_steelcity_2025pasc_1740095608642.csv")

# add new columns
df["totalTeleopCoral"] = df["teleopCoralL1"] + df["teleopCoralL2"] + df["teleopCoralL3"] + df["teleopCoralL4"]
df["totalAutoCoral"] = df["autoCoralL1"] + df["autoCoralL2"] + df["autoCoralL3"] + df["autoCoralL4"]

df["totalTeleopCoralPoints"] = df["teleopCoralL1"]*2 + df["teleopCoralL2"]*3 + df["teleopCoralL3"]*4 + df["teleopCoralL4"]*5
df["totalTeleopAlgaePoints"] = df["teleopAlgaeNet"]*4 + df["teleopAlgaeProc"]*6
df["totalTeleopPoints"] = df["totalTeleopCoralPoints"] + df["totalTeleopAlgaePoints"]

df["totalAutoCoralPoints"] = df["autoCoralL1"]*3 + df["autoCoralL2"]*4 + df["autoCoralL3"]*6 + df["autoCoralL4"]*7
df["totalAutoAlgaePoints"] = df["autoAlgaeNet"]*4 + df["autoAlgaeProc"]*6
df["totalAutoPoints"] = df["totalAutoCoralPoints"] + df["totalAutoAlgaePoints"]

df["algaeTeleop"] = df["teleopAlgaeNet"] + df["teleopAlgaeProc"]
df["algaeAuto"] = df["autoAlgaeNet"] + df["autoAlgaeProc"]

position = df["bargeStatus"]
df["endgamePoints"] = np.where(position == "Parked", 2, np.where(position == "Shallow Cage", 6, np.where(position == "Deep  Cage", 12, 0)))


df["totalPointsScored"] = df["totalTeleopPoints"] + df["totalAutoPoints"] + df["endgamePoints"]

# update team name
df["team_key"] = df["team_key"].str[3:]

# upcoming alliance lineup
red_teams = ["2172", "291", "3504"]
blue_teams = ["3260", "117", "4467"]
all_teams = red_teams + blue_teams

# filter df by team_key
new_df = df.loc[df["team_key"].isin(all_teams)]
new_df["colorGroup"] = new_df["team_key"].apply(lambda x: "Red" if x in red_teams else "Blue")

# averages df
averages_by_team = new_df.groupby("team_key").mean(numeric_only=True).reset_index()
averages_by_team_all = df.groupby("team_key").mean(numeric_only=True).reset_index()

teams = averages_by_team["team_key"]

# endgame df
endgame_df = new_df.groupby('team_key')['bargeStatus'].value_counts().unstack(fill_value=0).reset_index()

# color coding
ticktext = [
    f"<span style='color:{'red' if team in red_teams else 'blue'}'>{team}</span>"
    for team in all_teams
]

new_df = new_df.sort_values(["colorGroup", "team_key"], ascending=[True, True])

color_map = {str(team): "#FF5733" for team in red_teams}  # Red teams
color_map.update({str(team): "#1F77B4" for team in blue_teams})  # Blue teams

def color_picker(team_num):
            if team_num in red_teams:
                return "red"
            else:
                return "blue"

C:\Users\sienn\AppData\Local\Temp\ipykernel_25216\3695047634.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["colorGroup"] = new_df["team_key"].apply(lambda x: "Red" if x in red_teams else "Blue")


In [3]:
print(new_df)
print(averages_by_team)

      org_key  year event_key      match_key  match_number  \
4   steelcity  2025  2025pasc   2025pasc_qm1             1   
12  steelcity  2025  2025pasc   2025pasc_qm6             6   
3   steelcity  2025  2025pasc   2025pasc_qm1             1   
11  steelcity  2025  2025pasc   2025pasc_qm5             5   
6   steelcity  2025  2025pasc   2025pasc_qm2             2   
9   steelcity  2025  2025pasc   2025pasc_qm3             3   
0   steelcity  2025  2025pasc   2025pasc_qm1             1   
14  steelcity  2025  2025pasc   2025pasc_qm7             7   
1   steelcity  2025  2025pasc   2025pasc_qm1             1   
2   steelcity  2025  2025pasc   2025pasc_qm1             1   
18  steelcity  2025  2025pasc  2025pasc_qm12            12   

                     time alliance team_key  totalAutoPoints  \
4    2/15/2025 8:30:00 PM     blue      117                0   
12   2/15/2025 9:45:00 PM      red      117                0   
3    2/15/2025 8:30:00 PM     blue     3260               10   

In [4]:
print(endgame_df)

bargeStatus team_key  Deep Cage  Not Parked  Parked  Shallow Cage
0                117          0           1       1             0
1               2172          1           0       1             0
2                291          0           0       1             0
3               3260          0           1       0             1
4               3504          1           1       0             0
5               4467          0           2       0             0
